# Generate report from exprted photometer data  

Use raw folder with photometer exported data

## Imports

In [ ]:
VERBOSE_NOTEBOOK = False
WARNING_DISABLE = True
DEBUG = False

In [ ]:
from os import path
import warnings
from scipy.optimize import OptimizeWarning

if WARNING_DISABLE:
    warnings.simplefilter('ignore', RuntimeWarning)
    warnings.simplefilter('ignore', OptimizeWarning)
    warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')

In [ ]:
from mkinout import make_input_paths, parse_dir_name

WORKING_DIR = './reports/export/230712_AAV8-ELISA_sey_GN004240-048'
DATA_DIR = './data'

PARSED_DIR = parse_dir_name(WORKING_DIR)
WORKLIST_FILE_PATH = path.join(WORKING_DIR,  PARSED_DIR['date'] + '_worklist-ELISA.xls')
if not path.isfile(WORKLIST_FILE_PATH):
    raise Exception(f'Invalit worklist file {WORKLIST_FILE_PATH}!')

PARAMS_FILE_PATH = path.join(WORKING_DIR, PARSED_DIR['date'] + '_Parameters_' + PARSED_DIR['protocol'] + '.csv')
if not path.isfile(PARAMS_FILE_PATH):
    raise Exception(f'Invalid parameters file {PARAMS_FILE_PATH}!')

## Layouts

In [ ]:
from readdata import read_layouts

PLATE_LAYOUT_ID = 'plate_layout_ident.csv'
PLATE_LAYOUT_NUM = 'plate_layout_num.csv'
PLATE_LAYOUT_DIL_ID = 'plate_layout_dil_id.csv'


g_lay = read_layouts(path.join(DATA_DIR, PLATE_LAYOUT_ID),
                     path.join(DATA_DIR, PLATE_LAYOUT_NUM),
                     path.join(DATA_DIR, PLATE_LAYOUT_DIL_ID))

if VERBOSE_NOTEBOOK:
    display(g_lay)

## Worklist

In [ ]:
from worklist import read_worklist, check_worklist
from readdata import read_params

g_wl_raw = read_worklist(WORKLIST_FILE_PATH)
g_valid_plates = check_worklist(g_wl_raw)
g_params = read_params(PARAMS_FILE_PATH)

## Dilution to Concentration

Define dilution dataframe. The dataframe is indexed according plate layout, index of refference dataframe corresponds to refference of the `plate_layout_dil`.

In [ ]:
# TODO: read reference value from parameters
REF_VAL_MAX = 1.7954e+10
DILUTIONS = [1.0, 2.0, 4.0, 8.0, 16.0, 32.0, 64.0]

from sample import make_concentration
g_reference_conc = make_concentration(REF_VAL_MAX, DILUTIONS)

if VERBOSE_NOTEBOOK:
    display(g_reference_conc)

## Report generation

In [ ]:
from reportmain import report_plate, check_report_crc
from mkinout import make_output_paths, basename_from_inputdir, parse_dir_name
from datetime import datetime

def make_input_analysis_txt(work_dir, protocol, plate_id, dt):
    sdt = dt.strftime('%Y%m%d_%H%M%S')
    sd = dt.strftime('%y%m%d')
    f = f'{sd}_{protocol}__{plate_id}_{sdt}.txt'
    p = path.join(work_dir, f)
    if not path.isfile(p):
        raise Exception(f'Invalid plate export file {p}!')
    return p


def gen_report(valid_plates, worklist, params, layout, reference_conc,
               working_dir, base_name):
    reports = []
    for plate in valid_plates:
        print('Processing plate {} of {}'.format(plate, len(valid_plates)))

        output_files = make_output_paths(working_dir, base_name, plate)
        # TODO !!! FIXED DATETIME !!!
        dt = datetime(2023, 7, 12, 9, 7, 48)
        analysis_file_path = make_input_analysis_txt(working_dir, PARSED_DIR['protocol'],
                                                     plate, dt)
        report_file_path = output_files['report']
        # results_file_path = output_files['plate_results']

        report_dir = path.dirname(path.abspath(report_file_path))
        info = parse_dir_name(working_dir)
        md = report_plate(plate, worklist, params, layout,
                    reference_conc, analysis_file_path, report_dir, report_file_path,
                    info
                    )
        reports.append({'md': md, 'path': report_file_path})
    return reports

reports = gen_report(g_valid_plates, g_wl_raw, g_params, g_lay, g_reference_conc,
    WORKING_DIR, basename_from_inputdir(WORKING_DIR))

In [ ]:
CHECK_REPORT_CRC = True
REPORT_PLATES_CRC = [864111381, 3242056329]
if CHECK_REPORT_CRC:
    for report, crc in zip(reports, REPORT_PLATES_CRC):
        try:
            check_report_crc(report['md'], crc)
        except Exception as e:
            print('{} for {}'.format(e, report['path']))

Use pandoc to convert markdown to Word.

In [ ]:
PDFLATEX_EXE = 'c:/Users/hwn6193/AppData/Local/Programs/MiKTeX/miktex/bin/x64/pdflatex.exe'
REFERENCE_DOCX = 'C:/work/report-gen/custom-reference.docx'
for report in reports:
    report_file_path = path.abspath(report['path'])
    report_dir = path.dirname(path.abspath(report_file_path))
    docx_path = path.splitext(report_file_path)[0] + '.docx'
    print('Generating Word {} for {}'.format(docx_path, report_file_path))
    ! c:/work/pandoc/pandoc -o {docx_path} -f markdown -t docx --resource-path {report_dir} --reference-doc {REFERENCE_DOCX} {report_file_path}

    pdf_path = path.splitext(report_file_path)[0] + '.pdf'
    print(f'Generating PDF {pdf_path} for {report_file_path}')
    ! c:/work/pandoc/pandoc -s -o {pdf_path} --resource-path {report_dir} --pdf-engine {PDFLATEX_EXE} {report_file_path}

In [ ]:
# ! c:/work/pandoc/pandoc -o custom-reference.docx --print-default-data-file reference.docx